In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


data2022 = pd.read_csv('data.csv')
data2021 = pd.read_csv('data-2021.csv')
log2022 = pd.read_csv('log.csv')
log2021 = pd.read_csv('log-2021.csv')

In [24]:
def clean_data(data_df,log_df):

# Drop rows with missing final scores
  data_df = data_df[data_df['Score'] != '-']

# Convert numeric columns to appropriate data types
  numeric_columns = data_df.columns[data_df.columns.str.startswith('Week')]
  data_df[numeric_columns] = data_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Filter out irrelevant entries
  log_df = log_df[log_df['Score'] != '-']

# Merge data frames
  merged_df = pd.merge(data_df,log_df , on='Student')

# Filter out students with final grade A who did not submit homeworks from week 3 onwards
  merged_df['Zero_count'] = (merged_df.loc[:, 'Week 1 quiz':'Week 15 homework'] == 0).astype(int).sum(axis=1)
  passed_pre_exam = merged_df[(merged_df['Grade'] == 'A') & (merged_df['Zero_count'] >= 3)]
  student_list = passed_pre_exam['Student'].to_list()
  filtered_df = merged_df[(merged_df['Student'].isin(student_list)) == False]
  filtered_df.drop('Zero_count', axis='columns', inplace=True)
  filtered_df.reset_index(drop=True)
# Convert time column to datetime
  filtered_df['Time'] = pd.to_datetime(filtered_df['Time'])

# Calculate the deadline for each activity
  filtered_df['Deadline'] = filtered_df.apply(lambda row: row['Time'] + timedelta(days=4) if row['Day'] != 1 else row['Time'] + timedelta(days=6, hours=14), axis=1)

# Filter out attempts made after the deadline
  filtered_df = filtered_df[filtered_df['Time'] <= filtered_df['Deadline']]
  filtered_df.drop_duplicates(keep="first",inplace=True)
  filtered_df.reset_index(drop=True)
  return filtered_df
data_2022=clean_data(data2022,log2022)
data_2021=clean_data(data2021,log2021)